In [ ]:
pip install surprise

In [2]:
pwd

'C:\\Users\\suuser'

In [3]:
cd ./Desktop

C:\Users\suuser\Desktop


In [24]:
#İmporting the dataframe prepared for fine-tuning task
import pandas as pd
import numpy as np
df = pd.read_pickle("./final_df.pkl")  

In [25]:
df.head()

,overall,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,domain,...,description,title,also_buy,brand,feature,rank,also_view,date,price,details
0,5.0,"07 24, 2014",A320NYVVIZSZGY,0594481902,NaN,Colleen Marie,This AC adapter worked out just fine and as it...,Gives my NOOKY the juice it needs!,1406160000,elect,...,[#############################################...,Barnes &amp; Noble OV/HB Universal Power Kit f...,[],Barnes &amp; Noble,[Genuine Nook Charger + Genuine USB Charger Ca...,[>#55 in Electronics &gt; eBook Readers &amp; ...,"[059449771X, B00QZXQNNO, B014A9T98K, B00QZXQP5...","November 3, 2012",,NaN
1,5.0,"10 3, 2012",A21SESEJ8E52B7,0764207474,{'Format:': ' Kindle Edition'},tweezle,"When I looked at the cover art, I decided this...",Highly recommended!,1349222400,elect,...,[Two Desperate People--<BR>&#9;&#9;One With Al...,Not a Sparrow Falls,"[0764227289, 0764206680, 0764201670, 071808473...",Visit Amazon's Linda Nichols Page,[],"1,693,481 in Books (","[0764201670, 0764227289, 0764206680]",,$28.58,NaN
2,4.0,"05 20, 2010",A2JCJJNY43QQIV,0789743035,{'Format:': ' Paperback'},Katy Lake,this is a great idea. I know that virtually no...,If you've never owned a Macbook...,1274313600,elect,...,"[<b> </b>, , , ]",My MacBook,[],Visit Amazon's John Ray Page,[],"4,226,393 in Books (","[B001GN5OLK, B004FFVLYU, B07B7VFTN9]",,,NaN
3,3.0,"05 20, 2010",A3U029B8Z5WGI2,0789743035,{'Format:': ' Paperback'},KttyKat16,My MacBook is a good step-by-step manual on th...,"Nothing You Can't Find Online, but Easy to Fol...",1274313600,elect,...,"[<b> </b>, , , ]",My MacBook,[],Visit Amazon's John Ray Page,[],"4,226,393 in Books (","[B001GN5OLK, B004FFVLYU, B07B7VFTN9]",,,NaN
4,5.0,"11 21, 2012",A3TAS1AG6FMBQW,0972683275,NaN,Jeri Zerr,The VideoSecu line of articulating wall mounts...,"High quality construction, comes with plenty o...",1353456000,elect,...,[The videosecu TV mount is a mounting solution...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi...","[B000WYVBR0, B003O1UYHG, B002YV4WJS, B071HW7GS...",VideoSecu,"[Fits most 22"" to 47"" HDTV and some up to 55"" ...",[>#176 in Electronics &gt; Accessories &amp; S...,[],"February 25, 2007",$34.99,NaN


In [6]:
final_df = df

In [8]:
#Inspect duplicate rows
duplicate_rows = final_df[final_df.duplicated(subset='reviewText', keep=False)]
# duplicate rows
duplicate_rows.shape

#get rid of duplicate rows from final_df
final_df = final_df.drop_duplicates(subset='reviewText', keep='first')
# Reset the index and drop the old index column
final_df.reset_index(drop=True, inplace=True)

def sample_five_rows(reviewer_id, df, domains):
    sampled_rows = []
    for domain in domains:
        domain_df = df[(df['reviewerID'] == reviewer_id) & (df['domain'] == domain)].sample(n=5, random_state=3)
        sampled_rows.append(domain_df)
    return pd.concat(sampled_rows)

# Filter: to only include reviewerIDs that appear at least 5 times in each domain
reviewer_counts = final_df.groupby(['reviewerID', 'domain']).size().unstack(fill_value=0)
valid_reviewer_ids = reviewer_counts[(reviewer_counts['elect'] >= 5) & (reviewer_counts['clothes'] >= 5) & (reviewer_counts['home'] >= 5)].index

# Randomly select 50 unique reviewerIDs
selected_reviewer_ids = np.random.choice(valid_reviewer_ids, size=50, replace=False)

domains = ['elect', 'clothes', 'home']
sampled_dfs = []

for reviewer_id in selected_reviewer_ids:
    sampled_df = sample_five_rows(reviewer_id, final_df, domains)
    sampled_dfs.append(sampled_df)

subset_df = pd.concat(sampled_dfs)
# Reset the index and drop the old index column
subset_df.reset_index(drop=True, inplace=True)

In [9]:
subset_df['category'].value_counts()

[Electronics, Computers & Accessories, Tablet Accessories, Bags, Cases & Sleeves, Cases]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             11
[Electronics, Headphones, Earbud Headphones]                                                                                                                    

In [12]:
subset_df["overall"].value_counts()

5.0    420
4.0    194
3.0     73
2.0     33
1.0     30
Name: overall, dtype: int64

In [13]:
subset_df["title"][5]

"Invicta Men's 21501 Specialty Analog Display Swiss Quartz Silver-ToneWatch"

In [26]:
#Eliminating some unnecessary columns 
subset_df[['reviewerID', 'reviewerName', 'unixReviewTime', 'asin', 'style',
       'reviewText', 'overall', 'domain', 'category', 'title', 'brand',
       'date']].head()

,reviewerID,reviewerName,unixReviewTime,asin,style,reviewText,overall,domain,category,title,brand,date
0,A31BD4RXCON7QO,Michael A. Duvernois,1249344000,B000FVGM38,"{'Capacity:': ' 8 GB', 'Package Type:': ' Frus...",1. This is a basic 8GB USB 2.0 flash drive. Wo...,5.0,elect,"[Electronics, Computers & Accessories, Data St...",Transcend JetFlash V30-16 GB USB 2.0 Flash Dri...,Transcend,"November 1, 2008"
1,A31BD4RXCON7QO,Michael A. Duvernois,1447113600,B0161D1DVY,NaN,"I did not find the app to be useful to me, so ...",5.0,elect,"[Electronics, Headphones, Earbud Headphones]",Jabra Sport Pace Wireless Bluetooth Earbuds - ...,Jabra,
2,A31BD4RXCON7QO,Michael A. Duvernois,1318464000,B0042GFYLI,NaN,"First the downside, it's slow both in terms of...",5.0,elect,"[Electronics, Camera & Photo, Lenses, Camera L...",Sony Alpha SEL18200 E-mount 18-200mm F3.5-6.3 ...,Sony,"September 9, 2010"
3,A31BD4RXCON7QO,Michael A. Duvernois,1246233600,B001XUQPB4,{'Color:': ' Black'},"Though it's advertised as for 16"" laptops, my ...",5.0,elect,"[Electronics, Computers & Accessories, Laptop ...",Case Logic 16-Inch Security-Friendly Laptop B...,Case Logic,"<div class=""a-fixed-left-grid a-spacing-none"">..."
4,A31BD4RXCON7QO,Michael A. Duvernois,1226448000,B000CSOXTO,{'Product Packaging:': ' Standard Packaging'},"There are plenty of ""what were they thinking"" ...",5.0,elect,"[Electronics, GPS, Finders & Accessories, Spor...",Garmin GPSMAP 60CSx Handheld GPS Navigator (Di...,Garmin,


In [15]:
subset_df.columns

Index(['overall', 'reviewTime', 'reviewerID', 'asin', 'style', 'reviewerName',
       'reviewText', 'summary', 'unixReviewTime', 'domain', 'category',
       'description', 'title', 'also_buy', 'brand', 'feature', 'rank',
       'also_view', 'date', 'price', 'details'],
      dtype='object')

In [16]:
subset_df['domain'].value_counts()

elect      250
clothes    250
home       250
Name: domain, dtype: int64

In [17]:
subset_df['reviewerID'].value_counts()

A31BD4RXCON7QO    15
A249G4SVEWV9UX    15
A3GMRYV7VAYK5Q    15
AI8EMB0HUC6Z5     15
A244CRJ2QSVLZ4    15
A29BRRHE07UADR    15
A1WKQ94M45D8MG    15
A17BUUBOU0598B    15
A3LOGP4VA9DX46    15
A3IY316DRNF5F2    15
AW6J9IILGOZY3     15
A1XR58IRKP3HEZ    15
A3QDY9I0CNMD2W    15
A89ZZKNDFQH23     15
A3CBCVHIK3G76X    15
A1B6O7SAIYG2N0    15
AHRNDSKWH1YBT     15
A2065HBMYDXJ1S    15
A2HY9R5BVVWDXW    15
A1WH9Z5S1J1VA4    15
A2XX2A4OJCDNLZ    15
A3M9ASEEDBTJ7V    15
A22CW0ZHY3NJH8    15
A6X1O3A5C5FX1     15
APW0OINC8LISZ     15
A122SFGLAHQLSJ    15
AXR3CV6A4FMBG     15
A1EARN5PUVIF1S    15
A2I5LVVQ7WXEH6    15
A1HOLE9R6WPT85    15
A21U4DR8M6I9QN    15
A30X750M8V1XXW    15
A14B4MJ7KZE63B    15
A32NEDX6UVV8DE    15
A36WA7L88C2NW1    15
A35W3JQYP0M655    15
AK8LOFYRZJJW0     15
ATLA5CJH0ZZHP     15
A3M7BCQKSQAK4L    15
A2KETRDA2QWM8M    15
A3BZUQXA2QHDI2    15
ALNFHVS3SC4FV     15
A3F8T3BRXRX5P3    15
A276OI0NHBYORX    15
A3A5XNAM3ZW7Z6    15
A1PAGHECG401K1    15
A2Y739CRM15WDL    15
A2I85TDI7XF3C

In [21]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy
from collections import defaultdict

# Define the rating scale. For example, if 'overall' is a 5-point rating, the scale would be (1, 5)
reader = Reader(rating_scale=(1, 5))

# Load the data from the DataFrame
data = Dataset.load_from_df(subset_df[['reviewerID', 'asin', 'overall']], reader)

# Split the data into a train set and a test set
trainset, testset = train_test_split(data, test_size=0.2)

# Use the SVD algorithm
algo = SVD()

# Define a grid of parameters to search over
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}

# Use GridSearchCV to find the best parameters
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

# Fit the grid search to the data
gs.fit(data)

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
algo.fit(trainset)

# Test the trained model on the test set
predictions = algo.test(testset)

# Compute and print Root Mean Squared Error
print(f"Test set RMSE: {accuracy.rmse(predictions, verbose=True)}")

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions."""

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def get_actual(testset):
    """Return the actual items bought by each user in the test set."""

    actual = defaultdict(list)
    for uid, iid, true_r in testset:
        actual[uid].append((iid, true_r))

    return actual

top_n = get_top_n(predictions, n=5)
actual = get_actual(testset)

# Print the recommended items and predicted ratings for each user
for uid, user_ratings in top_n.items():
    print("Recommended:", uid, [(iid, round(est, 2)) for (iid, est) in user_ratings])
    print("Actual:", uid, [(iid, true_r) for (iid, true_r) in actual[uid]])



1.0414066630196122
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
RMSE: 1.0640
Test set RMSE: 1.0640492452797263
Recommended: A3QDY9I0CNMD2W [('B000096B01', 4.47), ('B002TECJPE', 4.35), ('B00CXMQ9HQ', 4.35), ('B00004RFIX', 4.35), ('B00ACIFZ50', 4.35)]
Actual: A3QDY9I0CNMD2W [('B000096B01', 5.0), ('B002TECJPE', 1.0), ('B00CXMQ9HQ', 3.0), ('B00004RFIX', 5.0), ('B00ACIFZ50', 5.0)]
Recommended: A1EARN5PUVIF1S [('B019Q88WOW', 4.29), ('B004VLVTRC', 4.29), ('B000I67QGC', 4.29)]
Actual: A1EARN5PUVIF1S [('B019Q88WOW', 4.0), ('B004VLVTRC', 5.0), ('B000I67QGC', 1.0)]
Recommended: A3A5XNAM3ZW7Z6 [('B0094S379U', 4.28), ('B00CKKB86S', 4.28), ('B00SH3ZSU4', 4.28)]
Actual: A3A5XNAM3ZW7Z6 [('B0094S379U', 4.0), ('B00CKKB86S', 1.0), ('B00SH3ZSU4', 4.0)]
Recommended: A3M9ASEEDBTJ7V [('B01CPZHPZK', 4.17), ('B01DTOHXMG', 4.17)]
Actual: A3M9ASEEDBTJ7V [('B01CPZHPZK', 3.0), ('B01DTOHXMG', 4.0)]
Recommended: A3M7BCQKSQAK4L [('B018QKJ6W8', 4.21), ('B0002CTETO', 4.21), ('B01FE2BXSG', 4.21)]
Actual: A3M7BCQKSQ